### Model: Significant Terms

Here, we model the recommendations using significant terms aggregation.

In [2]:
!pip install elasticsearch

The directory '/root/.cache/pip/http' or its parent directory is not owned by the current user and the cache has been disabled. Please check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.
The directory '/root/.cache/pip' or its parent directory is not owned by the current user and caching wheels has been disabled. check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.
    100% |################################| 102kB 235kB/s a 0:00:011
    100% |################################| 133kB 172kB/s ta 0:00:01
    100% |################################| 163kB 580kB/s ta 0:00:01


In [3]:
from elasticsearch import Elasticsearch
es = Elasticsearch(['elasticsearch'])

In [4]:
try:
    es.indices.delete(index='events')
    print('deleted')
except: 
    print('no index')

deleted


In [5]:
#Create mapping

es.indices.create(index='events', body = {
    "mappings": {
        "properties": {
            "target": {
                "type": "keyword"
            },
            "agent": {
                "type": "keyword"
            },
            "actions": {
                "type": "keyword"
            },
            "time": {
                "type": "date"
            }
            
        }
    }
})

{u'acknowledged': True, u'index': u'events', u'shards_acknowledged': True}

In [9]:
# (only run once) Here's the script to populate some qualitative data in the index
import random
def event_generator(num):
    ev_agents = ['neal','billg','sid','ethanh','donatello','leonardo','michaelangelo','raphael','mada','john','rolo','eminem']
    ev_actions = ['buy','referred_by:sid','view','referred_by:leonardo','referred_by:mada', 'returned']
    ev_targets = ['prod:1','prod:2','prod:3','prod:4','prod:5','prod:6','prod:9','prod:8','prod:7','prod:10','prod:11','prod:12']
    for n in range(1,num + 1 ):
        d1 = random.randint(0,5)
        d2 = random.randint(0,5)
        yield {"actions": [ random.choice(ev_actions), ev_actions[abs(d1-d2)] ],"target": random.choice(ev_targets), "agent": ev_agents[d1+d2]}
    
for ev in event_generator(1000):
    es.index(index='events', body=ev)


In [10]:
print(es.count(index='events'))
es.search(index='events')

{u'count': 1000, u'_shards': {u'successful': 1, u'failed': 0, u'skipped': 0, u'total': 1}}


{u'_shards': {u'failed': 0, u'skipped': 0, u'successful': 1, u'total': 1},
 u'hits': {u'hits': [{u'_id': u'U1E8bXIB_R6QZI8XI4P1',
    u'_index': u'events',
    u'_score': 1.0,
    u'_source': {u'actions': [u'referred_by:leonardo',
      u'referred_by:leonardo'],
     u'agent': u'ethanh',
     u'target': u'prod:6'},
    u'_type': u'_doc'},
   {u'_id': u'VFE8bXIB_R6QZI8XJINb',
    u'_index': u'events',
    u'_score': 1.0,
    u'_source': {u'actions': [u'view', u'referred_by:mada'],
     u'agent': u'michaelangelo',
     u'target': u'prod:11'},
    u'_type': u'_doc'},
   {u'_id': u'VVE8bXIB_R6QZI8XJINl',
    u'_index': u'events',
    u'_score': 1.0,
    u'_source': {u'actions': [u'view', u'referred_by:sid'],
     u'agent': u'ethanh',
     u'target': u'prod:12'},
    u'_type': u'_doc'},
   {u'_id': u'VlE8bXIB_R6QZI8XJINq',
    u'_index': u'events',
    u'_score': 1.0,
    u'_source': {u'actions': [u'referred_by:sid', u'buy'],
     u'agent': u'mada',
     u'target': u'prod:8'},
    u'_type':

In [15]:
# Most significant products by views
es.search(index='events', body={
    "query" : {
        "terms" : {"actions" : [ "view" ]}
    },
    "aggregations" : {
        "significant_products" : {
            "significant_terms" : { "field" : "target" }
        }
    }
} )

{u'_shards': {u'failed': 0, u'skipped': 0, u'successful': 1, u'total': 1},
 u'aggregations': {u'significant_products': {u'bg_count': 1000,
   u'buckets': [{u'bg_count': 86,
     u'doc_count': 38,
     u'key': u'prod:3',
     u'score': 0.02709834501856692},
    {u'bg_count': 87,
     u'doc_count': 35,
     u'key': u'prod:7',
     u'score': 0.013846991550489482},
    {u'bg_count': 68,
     u'doc_count': 28,
     u'key': u'prod:10',
     u'score': 0.013204598100505432},
    {u'bg_count': 95,
     u'doc_count': 36,
     u'key': u'prod:5',
     u'score': 0.007496290501953262},
    {u'bg_count': 79,
     u'doc_count': 28,
     u'key': u'prod:2',
     u'score': 0.00032141043513223065}],
   u'doc_count': 353}},
 u'hits': {u'hits': [{u'_id': u'VFE8bXIB_R6QZI8XJINb',
    u'_index': u'events',
    u'_score': 1.0,
    u'_source': {u'actions': [u'view', u'referred_by:mada'],
     u'agent': u'michaelangelo',
     u'target': u'prod:11'},
    u'_type': u'_doc'},
   {u'_id': u'VVE8bXIB_R6QZI8XJINl',
  

In [25]:
# Show products referred by sid
es.search(index='events', body={
    "query" : {
        "terms" : {"actions" : [ "referred_by:sid" ]}
    },
} )

{u'_shards': {u'failed': 0, u'skipped': 0, u'successful': 1, u'total': 1},
 u'hits': {u'hits': [{u'_id': u'VVE8bXIB_R6QZI8XJINl',
    u'_index': u'events',
    u'_score': 1.0,
    u'_source': {u'actions': [u'view', u'referred_by:sid'],
     u'agent': u'ethanh',
     u'target': u'prod:12'},
    u'_type': u'_doc'},
   {u'_id': u'VlE8bXIB_R6QZI8XJINq',
    u'_index': u'events',
    u'_score': 1.0,
    u'_source': {u'actions': [u'referred_by:sid', u'buy'],
     u'agent': u'mada',
     u'target': u'prod:8'},
    u'_type': u'_doc'},
   {u'_id': u'WFE8bXIB_R6QZI8XJINw',
    u'_index': u'events',
    u'_score': 1.0,
    u'_source': {u'actions': [u'referred_by:mada', u'referred_by:sid'],
     u'agent': u'leonardo',
     u'target': u'prod:2'},
    u'_type': u'_doc'},
   {u'_id': u'WVE8bXIB_R6QZI8XJINz',
    u'_index': u'events',
    u'_score': 1.0,
    u'_source': {u'actions': [u'buy', u'referred_by:sid'],
     u'agent': u'john',
     u'target': u'prod:4'},
    u'_type': u'_doc'},
   {u'_id': u'

In [27]:
# Most significant products referred by sid
es.search(index='events', body={
    "query" : {
        "terms" : {"actions" : [ "returned" ]}
    },
    "aggregations" : {
        "significant_products" : {
            "significant_terms" : { "field" : "target",
        
        "background_filter": {
                	"term" : { "actions" : "referred_by:sid"}
                }
        
    }
} } } )

{u'_shards': {u'failed': 0, u'skipped': 0, u'successful': 1, u'total': 1},
 u'aggregations': {u'significant_products': {u'bg_count': 405,
   u'buckets': [{u'bg_count': 30,
     u'doc_count': 22,
     u'key': u'prod:2',
     u'score': 0.03987247794567211},
    {u'bg_count': 24,
     u'doc_count': 18,
     u'key': u'prod:6',
     u'score': 0.03527600663813432},
    {u'bg_count': 37,
     u'doc_count': 22,
     u'key': u'prod:11',
     u'score': 0.012879680274212548},
    {u'bg_count': 36,
     u'doc_count': 20,
     u'key': u'prod:7',
     u'score': 0.004803913005502656},
    {u'bg_count': 36,
     u'doc_count': 20,
     u'key': u'prod:3',
     u'score': 0.004803913005502656}],
   u'doc_count': 214}},
 u'hits': {u'hits': [{u'_id': u'XFE8bXIB_R6QZI8XJIN7',
    u'_index': u'events',
    u'_score': 1.0,
    u'_source': {u'actions': [u'returned', u'referred_by:sid'],
     u'agent': u'leonardo',
     u'target': u'prod:2'},
    u'_type': u'_doc'},
   {u'_id': u'XlE8bXIB_R6QZI8XJIOA',
    u'_in